In [4]:
import numpy as np
import pandas as pd
from scipy import stats
from sklearn import metrics, preprocessing

In [9]:
def bin_values(xs, ys, interval=0.01, range_min=0.0, range_max=1.0, method='mean', thred_min_val_num=10):
    xs_bins = []
    ys_bins = []
    xs = np.array(xs)
    ys = np.array(ys)
    for start in np.arange(range_min, range_max, interval):
        values = ys[np.where((xs >= start) & (xs <= start+interval))]
        if len(values) < thred_min_val_num:
            continue
        xs_bins.append(start + 0.5 * interval)
        if method == 'mean':
            ys_bins.append(np.mean(values))
        elif method == 'median':
            ys_bins.append(np.median(values))
        else:
            ys_bins.append(np.mean(values))
    return np.array(xs_bins), np.array(ys_bins)

def bin_values_with_std(xs, ys, interval=0.01, range_min=0.0, range_max=1.0, method='mean', thred_min_val_num=10):
    xs_bins = []
    ys_bins = []
    std_bins = []
    for start in np.arange(range_min, range_max, interval):
        values = ys[np.where((xs >= start) & (xs <= start+interval))]
        if len(values) < thred_min_val_num:
            continue
        xs_bins.append(start + 0.5 * interval)
        if method == 'mean':
            ys_bins.append(np.mean(values))
        elif method == 'median':
            ys_bins.append(np.median(values))
        else:
            ys_bins.append(np.mean(values))
        std_bins.append(np.std(values))
    return np.array(xs_bins), np.array(ys_bins), np.array(std_bins)

def get_idx_without_outliers(data, m=3):
    valid_idx = np.where(abs(data - np.mean(data)) < m * np.std(data))
    return valid_idx

def get_df_all(year_list=[2002, 2005, 2008, 2011, 2014, 2017]):
    df_city_info = pd.read_csv('../data/df_city_info.csv')
    df_all = pd.DataFrame()
    for year in year_list:
        df_one_year = pd.read_csv('../data/df_{}.csv'.format(year))
        df_one_year['year'] = year
        df_all = pd.concat([df_all, df_one_year], axis=0)
    df_all = df_all[df_all['UI'] > 0].reset_index(drop=True)
    df_all = df_all.reset_index(drop=True)
    valid_idx_list = get_idx_without_outliers(df_all['EVI'], m=5)
    df_all = df_all.iloc[valid_idx_list].reset_index(drop=True)
    df_all['EVI'] = preprocessing.minmax_scale(df_all['EVI'])
    df_all = pd.merge(left=df_all, right=df_city_info[['city_id', 'cz', 'cz_name']], on='city_id', how='left')
    return df_all


In [10]:
def calc_ui_vi(ui_list, vi_list, interval=0.01, Vn=None):
    ui_bins_list, vi_bins_list = bin_values(ui_list, vi_list, interval=interval, thred_min_val_num=1)
    deg = 3
    reg = np.polyfit(ui_bins_list, vi_bins_list, deg=deg)
    x_fit_list = np.arange(np.min(ui_list), np.max(ui_list), 0.01)
    y_fit_list = np.polyval(reg, x_fit_list)
    vi_bins_list_fit = np.polyval(reg, ui_bins_list)
    r2 = metrics.r2_score(vi_bins_list, vi_bins_list_fit)
    print('R squre = {:.3f}'.format(r2))

    Vv_reg = np.polyval(reg, 0)
    V_obs_max = vi_bins_list[0]
    Vv = Vv_reg
    ui_max = np.max(ui_list)
    Vn_reg = np.polyval(reg, 1)
    if Vn is None:
        Vn = max(Vn_reg, 0.01)
    print('Vv = {:.3f}\tVn = {:.3f}'.format(Vv, Vn))
    
    vz_list = []
    v_diff_list = []
    for i in range(len(ui_list)):
        ui = ui_list[i]
        vz = (Vn - Vv) * ui + Vv
        vz_list.append(vz)
        v_diff_list.append(vi_list[i] - vz)
    vz_list = np.array(vz_list)
    v_diff_list = np.array(v_diff_list)

    wd_list = []
    wi_list = []
    ui_list_w = []
    for i in range(len(vz_list)):
        vz = vz_list[i]
        vobs = vi_list[i]
        if vz <= 0.001:
            continue
        wd = (vz - Vv) / Vv * 100
        wi = (vobs - vz) / vz * 100
        wd_list.append(wd)
        wi_list.append(wi)
        ui_list_w.append(ui_list[i])
    ui_list_w = np.array(ui_list_w)
    wd_list = np.array(wd_list)
    wi_list = np.array(wi_list)

    offset_list = []
    ui_list_offset = []
    for i in range(len(vz_list)):
        vz = vz_list[i]
        vobs = vi_list[i]
        if abs(Vv - vz) <= 0.001:
            continue
        offset = (vobs - vz) / (Vv - vz) * 100
        ui_list_offset.append(ui_list[i])
        offset_list.append(offset)
    offset_list = np.array(offset_list)
    ui_list_offset = np.array(ui_list_offset)
    
    if len(ui_list) < 500:
        interval = interval*5
    ui_bins_list_diff, v_diff_bins_list = bin_values(ui_list, v_diff_list, interval=interval, method='mean', thred_min_val_num=1)
    ui_bins_list_wi, wi_bins_list = bin_values(ui_list_w, wi_list, interval=interval, method='mean', thred_min_val_num=1)
    ui_bins_list_offset, offset_bins_list = bin_values(ui_list_offset, offset_list, interval=interval, method='mean', thred_min_val_num=1)
    valid_idx = get_idx_without_outliers(offset_bins_list, m=2)
    ui_bins_list_offset = ui_bins_list_offset[valid_idx]
    offset_bins_list = offset_bins_list[valid_idx]
    
    return ui_bins_list, vi_bins_list, x_fit_list, y_fit_list, Vv, Vn, v_diff_list, ui_list_w, wd_list, wi_list, ui_list_offset, offset_list, ui_bins_list_diff, v_diff_bins_list, ui_bins_list_wi, wi_bins_list, ui_bins_list_offset, offset_bins_list, p_val, r2

In [ ]:
# 计算每个城市的曲线，多年的Vn不变的算法 (把数据做了截断标准化，又改成了Vn取平均的方式)
year_list_ = []
city_name_list_ = []
loc_x_list = []
loc_y_list = []
cz_list = []
pnt_cnt_list = []
ui_mean_list = []
ndvi_mean_list = []
nirv_mean_list = []
offset_mean_ndvi_list = []
offset_mean_nirv_list = []
wi_mean_ndvi_list = []
wi_mean_nirv_list = []
wi_max_ndvi_list = []
wi_max_nirv_list = []
Vv_ndvi_list = []
Vv_nirv_list = []
Vn_ndvi_list = []
Vn_nirv_list = []
wd_mean_ndvi_list = []
wd_mean_nirv_list = []
wi_mean_ndvi_l_list = []
wi_mean_ndvi_m_list = []
wi_mean_ndvi_h_list = []
wi_mean_nirv_l_list = []
wi_mean_nirv_m_list = []
wi_mean_nirv_h_list = []
wi_mean_ndvi_ll_list = []
wi_mean_ndvi_hh_list = []
wi_mean_nirv_ll_list = []
wi_mean_nirv_hh_list = []
p_val_ndvi_list = []
r2_ndvi_list = []
p_val_nirv_list = []
r2_nirv_list = []

year_list = [2002, 2005, 2008, 2011, 2014, 2017]
df = get_df_all(year_list=year_list)
city_ids = list(np.unique(df['city_id']))

for city_id in city_ids:
    print('City ID: {}'.format(city_id))
    df_city_multiyear = df[df['city_id'] == city_id].reset_index(drop=True)
    
    for year in year_list:
        print('Year: {}'.format(year))
        df_city = df_city_multiyear[df_city_multiyear['year'] == year].reset_index(drop=True)
        try:
            ui_list = np.array(df_city['UI'])
            vi_evi_list = np.array(df_city['EVI'])
            valid_idx_ui_list = np.where(ui_list > 0.0)[0]
            valid_idx_evi_list = np.where(np.invert(np.isnan(vi_evi_list)))[0]
            valid_idx_evi_list = np.array(list(set(valid_idx_ui_list).intersection(set(valid_idx_evi_list))))

            ui_evi_list = ui_list[valid_idx_evi_list]
            vi_evi_list = vi_evi_list[valid_idx_evi_list]

            ui_bins_list, vi_bins_list, x_fit_list, y_fit_list, Vv, Vn, v_diff_list, ui_list_w, wd_list, wi_list, ui_list_offset, offset_list, ui_bins_list_diff, v_diff_bins_list, ui_bins_list_wi, wi_bins_list, ui_bins_list_offset, offset_bins_list, p_val, r2 = calc_ui_vi(ui_list=ui_evi_list, vi_list=vi_evi_list, interval=0.01, Vn=None)
            Vv_ndvi = Vv
            Vn_ndvi = Vn
            wd_mean_ndvi = (((Vv + Vn) / 2.0) - Vv) / Vv
            wi_mean_ndvi = np.mean(wi_bins_list)
            if wi_mean_ndvi >= 0:
                wi_max_ndvi = np.max(wi_bins_list)
            else:
                wi_max_ndvi = np.min(wi_bins_list)
            offset_mean_ndvi = np.mean(offset_bins_list)
            bins_len = len(wi_bins_list)
            wi_mean_ndvi_ll = np.mean(wi_bins_list[:int(bins_len/2.0)])
            wi_mean_ndvi_hh = np.mean(wi_bins_list[int(bins_len/2.0):])
            p_val_ndvi = p_val
            r2_ndvi = r2

        except:
            print('ERROR! <city_id: {}>'.format(city_id))
            continue
        
        loc_x = np.mean(df_city['lon'])
        loc_y = np.mean(df_city['lat'])
        cz = max(list(df_city['cz']), key=list(df_city['cz']).count)
        year_list_.append(year)
        city_name_list_.append(city_id)
        cz_list.append(cz)
        pnt_cnt_list.append(len(ui_evi_list))
        loc_x_list.append(loc_x)
        loc_y_list.append(loc_y)
        ui_mean_list.append(np.mean(ui_evi_list))
        ndvi_mean_list.append(np.mean(vi_evi_list))
        
        wi_mean_ndvi_list.append(wi_mean_ndvi)
        wi_max_ndvi_list.append(wi_max_ndvi)
        
        wd_mean_ndvi_list.append(wd_mean_ndvi)
        wi_mean_ndvi_ll_list.append(wi_mean_ndvi_ll)
        wi_mean_ndvi_hh_list.append(wi_mean_ndvi_hh)
        r2_ndvi_list.append(r2_ndvi)
    print()